In [ ]:
# Import.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Setup plotting params.
plt.rcParams['figure.figsize'] = (14, 10)
sns.set_theme()

%matplotlib inline

In [ ]:
# Constants.
DATA_PATH  = "../data/"
ASTN_FILE = DATA_PATH + "table1.csv"
SPECTRA_PATH = "../data/spectra/"
COL_NAMES = ["wavelength", "flux", "flux_err"]

# File list.
data_astn = pd.read_csv(ASTN_FILE, sep=',')

In [ ]:
# Get ranges of each wing. The ranges are arbitrarily defined as
#     [
#         [flux_min - WING_P1, flux_min - WING_P2],
#         [flux_min + WING_P2, flux_min + WING_P1]
#     ]
def get_wings(df, WING_P1 = 10, WING_P2 = 4):
    # Get wavelength with minimum flux.
    # NOTE. This could be improved with a Gaussian or Laplacian fit.
    flux_min = df["wavelength"][df["flux"].idxmin()]

    # Return wing ranges based on function parameters.
    return (
        (flux_min-WING_P1, flux_min-WING_P2),
        (flux_min+WING_P2, flux_min+WING_P1)
    )

# Normalize set x, return x_scaled and x's minima and maxima.
def scale(x):
    x_min = x.min()
    x_max = x.max()
    return ((x - x_min)/(x_max - x_min), x_min, x_max)

# De-scale x_scaled using previously obtained minima and maxima.
def descale(x_scaled, x_min, x_max):
    return (x_max - x_min) * x_scaled + x_min

# Get and store the target temperature and fluxes for all spectra inside the li-
#     mits defined by WING_P1 and WING_P2. The way these parameters are used is
#     described in the `get_wings(...)` function.
def get_row(in_row, WING_P1 = 10, WING_P2 = 4):
    # Open dataframe.
    df = pd.read_csv(
        SPECTRA_PATH + in_row["filename"],
        sep="\s+", header=None, names=COL_NAMES
    )

    # Get wings and cut data outside of them. This leaves 190 rows per file.
    w_lims = get_wings(df, WING_P1, WING_P2)
    df.drop(df[(
        ((w_lims[0][0]>df["wavelength"]) | (df["wavelength"]>w_lims[0][1])) &
        ((w_lims[1][0]>df["wavelength"]) | (df["wavelength"]>w_lims[1][1]))
    )].index, inplace=True)

    # TODO. Scale importance of each parameter based on its error. We should
    #       prioritize entires with a lower flux_err. Then, include the strength
    #       of this scaling as a parameter.

    # Create the output row.
    flux_list = df["flux"].reset_index(drop=True).transpose()
    out_row = {
        "filename"    : in_row["filename"],
        "temperature" : in_row["temperature"]
    }

    # Place the ordered fluxes in.
    for i, flux in flux_list.items():
        out_row["flux %03d" % i] = flux

    return out_row

# Get two quadratic fits from the data in a pandas dataframe, one for each wing.
# NOTE. Currently does not return anything about the fit quality.
def get_quadratic_fits(df):
    # Array to contain the results of the polynomial fits.
    fits  = []

    # Get wing limits.
    W_LIMS = get_wings(df)

    for wi in range(2):
        # Dictionary to contain the wavelength, flux, and flux error.
        wings = {}
        # Extract values
        for col in COL_NAMES:
            wings[col] = df[col][
                (df["wavelength"] >= W_LIMS[wi][0]) &
                (df["wavelength"] <= W_LIMS[wi][1])
            ].values

        # The errors are stddev and numpy takes in variance, so we square the
        #     values in that array.
        wings["flux_err"] = np.square(wings["flux_err"])

        # Fit the arrays.
        fits.append(np.poly1d(np.polynomial.polynomial.polyfit(
            wings["wavelength"],   # x.
            wings["flux"],         # y.
            2,                     # degree.
            w = wings["flux_err"], # y_err.
            rcond=None, full=False
        )[::-1]))

    return fits

In [ ]:
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# First approach: do one quadratic fit on each wing, then run all 6 parameters
#     through a linear regression model to predict the temperature.

# --+ Fit nine random files and plot.
fig, axes = plt.subplots(3, 3, sharey = True, sharex = True)
plt.rcParams['figure.figsize'] = (14, 10)
colors = ["#BAD9E9", "#CE5146"]
CUT_TYPE = 1 # 0 is only fits, 1 is whole curve.

# Pre-determined test set (very ugly distributions).
test_set = [
    "1705080048011933_ha.txt", "1611160044011083_ha.txt", "1707120021012623_ha.txt",
    "1706010031012623_ha.txt", "1705070106012833_ha.txt", "1706010031012623_ha.txt",
    "1608130026013533_ha.txt", "1702180032013013_ha.txt", "1704110041011683_ha.txt"
]

for pi in range(9):
    # Extract a random filename from the csv.
    TESTFILE = data_astn["filename"].sample(n=1).values[0]
    # TESTFILE = test_set[pi]

    # Create a dataframe from the txt file.
    df = pd.read_csv(SPECTRA_PATH + TESTFILE, sep="\s+", header=None, names=COL_NAMES)

    # Perform fits.
    fits = get_quadratic_fits(df)

    # Remove all data from df outside of the relevant region.
    W_LIMS = get_wings(df)

    if CUT_TYPE == 0:
        df.drop(df[(
            ((W_LIMS[0][0] > df["wavelength"]) | (df["wavelength"] > W_LIMS[0][1])) &
            ((W_LIMS[1][0] > df["wavelength"]) | (df["wavelength"] > W_LIMS[1][1]))
        )].index, inplace=True)
    if CUT_TYPE == 1:
        df.drop(df[(
            (W_LIMS[0][0]-5 > df["wavelength"]) | (df["wavelength"] > W_LIMS[1][1]+5)
        )].index, inplace=True)

    # Plot df.
    axes[int(pi/3)][pi%3].errorbar(
        df["wavelength"], df["flux"], yerr=df["flux_err"],
        fmt = 'o', markersize=0.4, capsize=0.2, color=colors[0]
    )

    # Get two linspaces to plot the fit.
    for wi in range(2):
        x_vals = np.linspace(W_LIMS[wi][0], W_LIMS[wi][1], 100)
        y_vals = fits[wi](x_vals)
        axes[int(pi/3)][pi%3].plot(x_vals, y_vals, color=colors[1])

    # Print title.
    axes[int(pi/3)][pi%3].set_title(TESTFILE)

plt.tight_layout()
# plt.savefig("NAME.png")

print("Quadratic examples:")
plt.show()

# --+ Get and store the fits and the target temperature for all files.
# Create the output dataframe.
columns = ["filename", "w1_x2", "w1_x1", "w1_c", "w2_x2", "w2_x1", "w2_c", "temperature"]
out_df = pd.DataFrame(columns=columns)

# Iterate through all files.
for in_row in data_astn.iterrows():
    # Get the pandas dataframe.
    in_df = pd.read_csv(
        SPECTRA_PATH + in_row[1]["filename"], sep="\s+", header=None, names=COL_NAMES
    )

    # Extract the fits.
    fits = get_quadratic_fits(in_df)

    # Form the row.
    out_row = {
        "filename"    : in_row[1]["filename"],
        "w1_x2"       : fits[0][0],
        "w1_x1"       : fits[0][1],
        "w1_c"        : fits[0][2],
        "w2_x2"       : fits[1][0],
        "w2_x1"       : fits[1][1],
        "w2_c"        : fits[1][2],
        "temperature" : in_row[1]["temperature"]
    }

    # Append row to output dataframe.
    out_df.loc[in_row[0]] = out_row

# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Pairplot to look at correlations. Very high correlation between each quadratic
#     fit parameter. We'd expect one parameter to be superfluous (due to formula
#     ax^2 + bx + c = 0), but not all of them. Weird.
sns.pairplot(out_df)

print("\nPairplot:")
plt.show()

# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Prepare X_train, X_test, y_train, y_test.
X = out_df[["w1_x2", "w1_x1", "w1_c", "w2_x2", "w2_x1", "w2_c"]]
y = out_df["temperature"]

# Scale X and y. This is not necessary for linear regression, but might as well
#     do it now instead of forgetting to do it if we try more complex models in
#     the future. Also this scaling method is pretty bad, should use MinMax.
X_scalers = [x.max() if x.max() > abs(x.min()) else abs(x.min()) for _, x in X.items()]
y_scaler  = y.max() # y is strictly positive.

X_scaled = X / X_scalers
y_scaled = y / y_scaler

# Split of 80:20.
cutoff = int(.80 * X_scaled.shape[0])

X_train = X_scaled[:cutoff]
X_test  = X_scaled[cutoff:]
y_train = y_scaled[:cutoff]
y_test  = y_scaled[cutoff:]

# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Train lr models with different number of polynomial features.
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.preprocessing import PolynomialFeatures

r2_scores = {}
y_preds   = {}

for d in range(1, 11):
    # Get polynomial features.
    p2r = PolynomialFeatures(degree = d)
    X_train_poly = p2r.fit_transform(X_train)
    X_test_poly  = p2r.transform(X_test)

    # Fit and predict w/ linear regression.
    lr = LinearRegression()
    lr.fit(X_train_poly, y_train)

    y_preds[d]   = lr.predict(X_test_poly)
    r2_scores[d] = r2_score(y_preds[d], y_test)

# Negative R2 scores make sense considering the high degree of the polynomial
#     features. Suggests that a dense network might give good results, but we'd
#     need more data.
print("\nR^2 scores:")
for d in range(1, 11):
    print("  * polyfit (%d): %8.5f" % (d, r2_scores[d]))

# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
palette = ["#96D5A4", "#FA9A58", "#ED6345", "#CF384D"]

# Make lineplots.
sns.lineplot(y_scaler * y_test.values, color = palette[0], label = "Real values")
for d in range(1, 4):
    sns.lineplot(y_scaler * y_preds[d], color = palette[d], label = "Polyfit (%d)" % d, dashes=(2,2))

plt.ylim(5000, 7500)

# Set labels and such.
plt.ylabel("T_eff")
plt.legend(loc = "upper right")
# plt.savefig("NAME.png")

print("\nPolynomial fits results:")
plt.show()

In [ ]:
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Second approach: Perform a PCA decomposition on all data points in the wings.
#     Then, run all PCA components through a linear regression model to predict
#     the temperature.

# Run a linear regression model on a PCA decomposition of the data points on the
#     2 wings of a set of flux spectra files. The process uses three hyperpar-
#     ameters:
#       * WING_P1     : Wing maximum distance from distribution minimum.
#       * WING_P2     : Wing minimum distance from distribution minimum.
#       * N_PCA_COMPS : Number of PCA components to extract.
# Returns a tuple with mean absolute error, mean squared error, and R2 score.
def linreg_Teff(WING_P1=10, WING_P2=4, N_PCA_COMPS=20, plot=False, debug=False):
    # --+ Create the output dataframe.
    column_list = list(get_row(data_astn.loc[0], WING_P1, WING_P2).keys())
    out_df = pd.DataFrame(columns=column_list)

    for in_row in data_astn.iterrows():
        out_df.loc[in_row[0]] = get_row(in_row[1], WING_P1, WING_P2)

    # Remove columns with NaN values.
    out_df = out_df.dropna(axis = 1)
    
    # --+ Prepare X_train, X_test, y_train, y_test
    # Get X and y.
    X = out_df.drop(["filename", "temperature"], axis=1)
    y = out_df["temperature"]

    # Scale.
    (X_scaled, X_min, X_max) = scale(X)
    (y_scaled, y_min, y_max) = scale(y)

    # Split 80:20.
    cutoff = int(.80 * X_scaled.shape[0])

    X_train = X_scaled[:cutoff]
    X_test  = X_scaled[cutoff:]
    y_train = y_scaled[:cutoff]
    y_test  = y_scaled[cutoff:]

    # --+ Perform PCA.
    pca = PCA(n_components = N_PCA_COMPS)
    X_train = pca.fit_transform(X_train)
    X_test  = pca.transform(X_test)

    if (debug):
        print("PCA explained variance ratios:")
        ratios = pca.explained_variance_ratio_
        for i in range(len(ratios)):
            print("  * PCA param %2d: %6.4f" % (i, ratios[i]))

    # --+ Train lr model.
    # Fit and predict w/ linear regression.
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    y_pred = lr.predict(X_test)

    # --+ Plot.
    if plot:
        palette = ["#96D5A4", "#FA9A58"]

        # Make lineplots.
        sns.lineplot(
            descale(y_test.values, y_min, y_max),
            color = palette[0], label = "Real values"
        )
        sns.lineplot(
            descale(y_pred, y_min, y_max),
            color = palette[1], label = "Linear regression", dashes = (2,2)
        )

        # Set labels and such.
        plt.ylabel("T_eff")
        plt.legend(loc = "upper right")
        plt.ylim(5000, 7500)

        # plt.savefig("NAME.png")
        print("\nLinear regression result:")
        plt.show()

    return (
        mean_absolute_error(y_test, y_pred),
        mean_squared_error( y_test, y_pred),
        r2_score(           y_test, y_pred)
    )

In [ ]:
res_df = pd.DataFrame(columns=("wing r_max", "wing r_min", "n pca components", "mae", "mse", "R2 score"))

for WING_P1 in np.linspace(2, 12, 21):
    for WING_P2 in np.linspace(0, 10, 21):
        if WING_P1 < WING_P2:
            continue

        for N_PCA_COMPS in np.arange(2, 12, 2):
            try:
                res = linreg_Teff(WING_P1, WING_P2, N_PCA_COMPS)
            except ValueError: # On extreme cases, we won't get enough data points for doing PCA.
                continue

            res_row = pd.DataFrame({
                "wing r_max"       : [WING_P1],
                "wing r_min"       : [WING_P2],
                "n pca components" : [N_PCA_COMPS],
                "mae"              : [res[0]],
                "mse"              : [res[1]],
                "R2 score"         : [res[2]]
            })

            # Append row to output dataframe.
            res_df = pd.concat([res_df, res_row], ignore_index=True)

In [ ]:
# Check best performing PCA numbers.
res_df_npc = res_df.groupby("n pca components").mean().reset_index()

fig, axes = plt.subplots(3, 1, sharey = False, sharex = True)
plt.rcParams['figure.figsize'] = (14, 10)

metrics = ("mae", "mse", "R2 score")

for pi in range(3):
    axes[pi].plot(res_df_npc["n pca components"], res_df_npc[metrics[pi]])
    axes[pi].set_title(metrics[pi])

plt.tight_layout()
plt.show()

In [ ]:
best_results = {
    "mae"      : pd.DataFrame(columns=("wing r_max", "wing r_min", "n pca components", "mae", "mse", "R2 score")),
    "mse"      : pd.DataFrame(columns=("wing r_max", "wing r_min", "n pca components", "mae", "mse", "R2 score")),
    "R2 score" : pd.DataFrame(columns=("wing r_max", "wing r_min", "n pca components", "mae", "mse", "R2 score"))
}

for WING_P1 in np.linspace(2, 12, 21):
    for WING_P2 in np.linspace(0, 10, 21):
        df = res_df.loc[(res_df["wing r_max"] == WING_P1) & (res_df["wing r_min"] == WING_P2)]

        for metric in metrics:
            if df[metric].empty: continue
            idx = df[metric].idxmax() if metric == "R2 score" else df[metric].idxmin()
            row = df.loc[[idx]]
            best_results[metric] = pd.concat([best_results[metric], row], ignore_index=True)

In [ ]:
with sns.axes_style("white"):
    for metric in metrics:
        # Make plot.
        heatmap_data = best_results[metric].pivot(index="wing r_max", columns="wing r_min", values=metric)
        sns.heatmap(heatmap_data, linewidth=.5, annot=True, annot_kws={"size": 9})

        # Show.
        plt.title(metric)
        plt.savefig("heatmap_%s.png" % metric)
        plt.show()

In [ ]:
# TODO. For the best model, see how robust it is if we change the train set. Then, test different train set sizes.